In [1]:
from scipy.io import loadmat
from scipy.stats import circmean, circstd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

from utils import validate_data

## Loading all the data

In [2]:
# Set in stone
data_path = '../axej/'
n_subjects = 13
n_sessions = 4
n_runs = 1
n_trials = 120
n_ts = 500

exp_ts = 1000

# noise frames = 0 to 220 (250 - 30)
noise_thresh = 0.5
noise_gap = 30


experiment_orientations = [159, 123, 87, 51, 15]
subjects = ["01", "02", "03", "04", "05", "06", "07", "08" ,"09", "10", "11", "12", "14"]

def get_calib(subj, sess):
    mat_contents = loadmat(data_path + f'AxeJEEG_Subj{subjects[subj]}_S{sess+1}_Cali1.mat.mat', struct_as_record=False, squeeze_me=True)
    return mat_contents["p"].__dict__

def get_run(subj, sess, run):
    mat_contents = loadmat(data_path + f'AxeJEEG_Subj{subjects[subj]}_S{sess+1}_Run{run+1}.mat.mat', struct_as_record=False, squeeze_me=True)
    return mat_contents["p"].__dict__

In [3]:
# Get all run data
# subjexts x sessions x runs
jx = []
jy = []
stimdir = []
tgonset = []

# Attention (attCue): tr_foc = 1, tr_div = 2
# Coherence (tgCoh): tr_lo = 1, tr_hi = 2
att = []
coh = []

for subj in tqdm(range(n_subjects)):
    for sess in range(n_sessions):
        for run in range(n_runs):

            data = get_calib(subj, sess)
            jx.append(data["joyx"])
            jy.append(data["joyy"])
            stimdir.append(data["stimDirREAL"])

            f_tgonset = data["f_precuedur"] + data["f_cuedur"]
            tgonset.append(f_tgonset)

            att.append(data["attCue"])
            coh.append(data["tgCoh"])

# Shape the run data
jx = np.array(jx, dtype=np.float64).reshape(n_subjects, n_sessions, n_runs, n_trials, n_ts)
jy = np.array(jy, dtype=np.float64).reshape(n_subjects, n_sessions, n_runs, n_trials, n_ts)
stimdir = np.array(stimdir).reshape(n_subjects, n_sessions, n_runs, n_trials)
tgonset = np.array(tgonset).reshape(n_subjects, n_sessions, n_runs, n_trials)
att = np.array(att).reshape(n_subjects, n_sessions, n_runs, n_trials)
coh = np.array(coh).reshape(n_subjects, n_sessions, n_runs, n_trials)

jx.shape, jy.shape, stimdir.shape, tgonset.shape

100%|██████████| 13/13 [00:00<00:00, 47.17it/s]


((13, 4, 1, 120, 500), (13, 4, 1, 120, 500), (13, 4, 1, 120), (13, 4, 1, 120))

## Logic for Getting Valid Trials:

1. Get all the trials
2. Align all target onset at 250 ts
3. Get distance from center and angle from center
4. If the distance moves <0.4 au n frames before target onset. Then that trial is "too early" and all response set nan.
5. If ts has distance > 1 au then set its response angle to the last valid angle (if first, the nan) and set that distance to 1.
6. get angle first distance = 1 a.u. instance after target onset. or if it never reaches 1, then angle at max distance after target onset. as the response angle.

In [4]:
shifted_jx, shifted_jy, dist_from_cent, resp_angle, final_resp_angles = validate_data(n_subjects, n_sessions, n_runs, n_trials, n_ts, exp_ts, jx, jy, tgonset, noise_thresh, noise_gap)

In [5]:
# Get all trials where angles are only nan
nan_trials = np.where(np.isnan(resp_angle).all(axis=-1))
print(nan_trials[0].shape[0], "removed trials")
print(nan_trials[0].shape[0] / (n_subjects * n_sessions * n_runs * n_trials) * 100, "% removed trials")

197 removed trials
3.157051282051282 % removed trials


## Getting final responses

In [6]:
def circdist(a, b):
    return min(abs(a-b), 360-abs(a-b))

def circmedian(angs):
    angs = np.array(angs)
    angs = angs[~np.isnan(angs)]
    pdists = angs[np.newaxis, :] - angs[:, np.newaxis]
    pdists = (pdists + 180) % (2 * 180) - 180
    pdists = np.abs(pdists).sum(1)
    return angs[np.argmin(pdists)]

In [7]:
final_resp_angles.shape, stimdir.shape, np.unique(stimdir)

((13, 4, 1, 120),
 (13, 4, 1, 120),
 array([ 15,  51,  87, 123, 159], dtype=uint8))

In [8]:
# Group according to different stimdirs
# 5 angles in total and each subject has 4 * 120/ 5 = 96 trials per angle
stim_resp = np.zeros((n_subjects, 5, 96))

for sub in range(n_subjects):
    # for each unique stimdir
    for i, unique_stim in enumerate(np.unique(stimdir)):
        # get the response angle for that stimdir by masking the stimdir with the unique stimdir
        stim_resp[sub, i] = final_resp_angles[sub][stimdir[sub]==unique_stim]

stim_resp.shape

(13, 5, 96)

In [9]:
circ_median = np.zeros((n_subjects, 5))
circ_std = np.zeros((n_subjects, 5))

for sub in range(n_subjects):
    for i, unique_stim in enumerate(np.unique(stimdir)):
        circ_median[sub, i] = circmedian(stim_resp[sub, i])
        circ_std[sub, i] = np.rad2deg(circstd(np.deg2rad(stim_resp[sub, i]), nan_policy='omit'))

In [10]:
# Manual tweaking one to negatives
circ_median[11,0] = circ_median[11,0] - 360

In [11]:
pd.DataFrame(circ_median, index=subjects, columns=np.unique(stimdir))

,15,51,87,123,159
01,18.752776,32.872185,64.547658,138.718044,145.877462
02,36.980188,46.453086,69.256176,134.912042,141.796043
03,15.590052,55.295314,83.992805,124.461200,150.389078
04,15.891056,49.407544,89.413314,116.616832,148.362786
05,23.332555,35.213999,74.428780,142.623388,153.771178
06,19.011896,39.561438,67.231716,127.133737,150.723443
07,22.130973,38.140917,82.005264,127.348362,139.820962
08,16.351909,37.877136,82.692124,122.602734,143.130102
09,1.391104,37.853070,75.131218,125.861780,159.547839
10,23.204631,43.472421,90.349254,127.499126,144.594361


In [12]:
pd.DataFrame(circ_std, index=subjects, columns=np.unique(stimdir))

,15,51,87,123,159
01,9.022259,6.036654,11.451835,11.305999,5.991948
02,13.999590,9.031298,10.230571,6.295155,6.187493
03,11.178859,16.711538,10.886672,7.247380,14.853936
04,11.094986,9.799019,5.499574,9.622370,11.825942
05,11.053636,11.446332,11.261143,12.297227,9.700801
06,9.045171,8.031856,8.998997,7.385475,8.687619
07,9.529510,10.025275,8.359096,16.828024,9.591504
08,8.426546,8.591857,9.766157,8.657439,9.645553
09,4.381122,9.617331,7.500529,7.314267,10.317194
10,11.450448,16.236299,12.029523,10.773463,11.196595


In [14]:
# np.save('circ_median.npy', circ_median)
# np.save('circ_std.npy', circ_std)